# Recipe 2: Estimating Behavioral Response to Reform

This is an advanced recipe that should be followed only after mastering the basic recipe.
This recipe shows how to analyze the behavioral responses to a tax reform using the Behavioral-Responses behresp package.

In [ ]:
# Install conda and taxcalc if in Google Colab.
import sys
if 'google.colab' in sys.modules and 'taxcalc' not in sys.modules:
    !wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
    # Append path to be able to run packages installed with conda
    # This must correspond to the conda Python version, which may differ from
    # the base Colab Python installation.
    sys.path.append('/usr/local/lib/python3.7/site-packages')
    # Install PSL packages from Anaconda
    !yes | conda install -c PSLmodels taxcalc behresp

In [1]:
import taxcalc as tc
import behresp

# use publicly-available CPS input file
recs = tc.Records.cps_constructor()

# specify baseline Calculator object representing current-law policy
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

CYR = 2020

# calculate aggregate current-law income tax liabilities for cyr
calc1.advance_to_year(CYR)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

# specify Calculator object for static analysis of reform policy
pol.implement_reform(tc.Policy.read_json_reform('_static/reformA.json'))
calc2 = tc.Calculator(policy=pol, records=recs)

# calculate reform income tax liabilities for cyr under static assumptions
calc2.advance_to_year(CYR)
calc2.calc_all()
itax_rev2sa = calc2.weighted_total('iitax')

# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.25}

# specify Calculator object for analysis of reform with behavioral responses
calc2 = tc.Calculator(policy=pol, records=recs)
calc2.advance_to_year(CYR)
_, df2br = behresp.response(calc1, calc2, response_elasticities)

# calculate reform income tax liabilities for CYR with behavioral response
itax_rev2br = (df2br['iitax'] * df2br['s006']).sum()

# print total income tax revenue estimates for CYR
# (estimates in billons of dollars)
print('{}_CURRENT_LAW_P__itax_rev($B)= {:.3f}'.format(CYR, itax_rev1 * 1e-9))
print('{}_REFORM_STATIC__itax_rev($B)= {:.3f}'.format(CYR, itax_rev2sa * 1e-9))
print('{}_REFORM_DYNAMIC_itax_rev($B)= {:.3f}'.format(CYR, itax_rev2br * 1e-9))

2020_CURRENT_LAW_P__itax_rev($B)= 1430.743
2020_REFORM_STATIC__itax_rev($B)= 1423.661
2020_REFORM_DYNAMIC_itax_rev($B)= 1413.293


Create multi-year diagnostic tables for
1. baseline,
2. reform excluding behavioral responses, and
3. reform including behavioral responses

In [2]:
NUM_YEARS = 3  # number of diagnostic table years beginning with CYR
dtable1 = calc1.diagnostic_table(NUM_YEARS)
dtable2 = calc2.diagnostic_table(NUM_YEARS)
dvar_list3 = list()
year_list3 = list()
for year in range(CYR, CYR + NUM_YEARS):
    calc1.advance_to_year(year)
    calc2.advance_to_year(year)
    _, df2br = behresp.response(calc1, calc2, response_elasticities)
    dvar_list3.append(df2br)
    year_list3.append(year)
dtable3 = tc.create_diagnostic_table(dvar_list3, year_list3)

Diagnostic table for baseline:

In [3]:
dtable1

,2020,2021,2022
Returns (#m),167.510,169.750,171.930
AGI ($b),12081.707,12624.263,13145.018
Itemizers (#m),33.100,33.800,34.690
Itemized Deduction ($b),935.939,967.582,1016.742
Standard Deduction Filers (#m),134.410,135.950,137.240
Standard Deduction ($b),2382.421,2463.196,2543.217
Personal Exemption ($b),0.000,0.000,0.000
Taxable Income ($b),9264.215,9713.414,10129.576
Regular Tax ($b),1588.955,1673.211,1752.027
AMT Income ($b),11415.741,11935.474,12417.671


Diagnostic table for reform, excluding behavioral responses:

In [4]:
dtable2

,2020,2021,2022
Returns (#m),167.510,169.750,171.930
AGI ($b),12081.707,12624.263,13145.018
Itemizers (#m),33.030,33.730,34.610
Itemized Deduction ($b),933.241,965.027,1013.815
Standard Deduction Filers (#m),134.480,136.020,137.320
Standard Deduction ($b),2383.809,2464.550,2544.827
Personal Exemption ($b),333.334,344.504,356.215
Taxable Income ($b),9011.004,9451.253,9858.636
Regular Tax ($b),1579.369,1664.122,1743.689
AMT Income ($b),11418.092,11937.685,12420.167


Diagnostic table for reform, including behavioral responses:

In [5]:
dtable3

,2020,2021,2022
Returns (#m),167.510,169.750,171.930
AGI ($b),12059.423,12599.813,13118.735
Itemizers (#m),32.990,33.700,34.580
Itemized Deduction ($b),931.927,963.754,1012.542
Standard Deduction Filers (#m),134.530,136.050,137.350
Standard Deduction ($b),2384.573,2465.148,2545.379
Personal Exemption ($b),333.334,344.504,356.215
Taxable Income ($b),8988.663,9426.789,9832.403
Regular Tax ($b),1569.117,1653.068,1731.937
AMT Income ($b),11396.832,11914.300,12394.970
